### Export ONNX

ONNX tạo export đồ thị độc lập với phần cứng, tức là dù export trên CPU hay GPU thì graph được tạo ra cũng không đổi. ONNXRuntime mới tối ưu phụ thuộc vào phần cứng, đây là nơi tạo ra sự khác biệt.

In [1]:
import torch
from torchvision.models import resnet50

model = resnet50(pretrained=True)
model = model.eval()

/home/rmits/miniconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rmits/miniconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
ONNX_PATH = "resnet50.onnx"
IMG_SIZE = 224
OPSET = 18

In [4]:
dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE)
torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    opset_version=OPSET,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes=None, # Optional: specify dynamic axes if needed
    external_data=False
)

W0212 14:19:06.354000 947721 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0212 14:19:06.356000 947721 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0212 14:19:06.357000 947721 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.
W0212 14:19:06.358000 947721 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_sca

[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/home/rmits/miniconda3/envs/dl/lib/python3.10/copyreg.py:101: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 106 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.10.0+cu128',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[1,1000]>
            ),
            initializers=(
                %"conv1.weight"<FLOAT,[64,3,7,7]>{Tensor(...)},
                %"layer1.0.conv1.weight"<FLOAT,[64,64,1,1]>{Tensor(...)},
                %"layer1.0.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.0.conv3.weight"<FLOAT,[256,64,1,1]>{Tensor(...)},
                %"layer1.0.downsample.0.weight"<FLOAT,[256,64,1,1]>{Tensor(...)},
                %"layer1.1.conv1.weight"<FLOAT,[64,256,1,1]>{Tensor(...)},
                %"layer1.1.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
    

### Evaluate:

Đánh giá trên các EP khác nhau.

In [5]:
import onnxruntime as ort
from tqdm.notebook import tqdm
import numpy as np
import time

SAMPLES = 1000
IMG_SIZE = 224

In [6]:
def eval_fn(session):
    print(f"Evaluating ONNX model with providers: {session._providers}")
    inference_times = []
    for _ in tqdm(range(SAMPLES), desc=f"Running ONNX Inference: "):
        input_data = np.random.randn(1, 3, IMG_SIZE, IMG_SIZE).astype(np.float32)
        start_time = time.perf_counter()
        __ = session.run(['output'], {'input': input_data})
        inference_times.append((time.perf_counter() - start_time) * 1e3) # Convert to milliseconds
    # Calculate statistics
    mean_time = np.mean(inference_times)
    std_time = np.std(inference_times)
    min_time = np.min(inference_times)
    max_time = np.max(inference_times)
    median_time = np.median(inference_times)
    p95_time = np.percentile(inference_times, 95)
    p99_time = np.percentile(inference_times, 99)

    print(f"Inference Time Statistics (milliseconds):")
    print(f"  Mean: {mean_time:.6f}")
    print(f"  Std: {std_time:.6f}")
    print(f"  Min: {min_time:.6f}")
    print(f"  Max: {max_time:.6f}")
    print(f"  Median: {median_time:.6f}")
    print(f"  95th percentile: {p95_time:.6f}")
    print(f"  99th percentile: {p99_time:.6f}")
    print(f"\nThroughput: {SAMPLES / (sum(inference_times) / 1e3):.2f} samples/sec")

#### CPUExcutionProvider:

In [7]:
session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options.intra_op_num_threads = 4  # giá trị mặc định là 0 (tự động chọn số luồng dựa trên số lõi CPU)
session = ort.InferenceSession(ONNX_PATH, sess_options=session_options, providers=['CPUExecutionProvider'])
eval_fn(session)

Evaluating ONNX model with providers: ['CPUExecutionProvider']


Running ONNX Inference:   0%|          | 0/1000 [00:00<?, ?it/s]

Inference Time Statistics (milliseconds):
  Mean: 19.507692
  Std: 2.172896
  Min: 18.600732
  Max: 51.288857
  Median: 19.110651
  95th percentile: 20.653843
  99th percentile: 29.535244

Throughput: 51.26 samples/sec


#### CUDAExecutionProvider:

Nếu thiếu cudnn, cài đặt bằng: `conda install cudnn=9 cuda=12 -c nvidia`

In [9]:
session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session = ort.InferenceSession(
    ONNX_PATH, 
    sess_options=session_options, 
    providers = [
        ("CUDAExecutionProvider", {
            "device_id": 0,
            "arena_extend_strategy": "kNextPowerOfTwo",
            "cudnn_conv_algo_search": "EXHAUSTIVE",
            "enable_cuda_graph": True, # Enable CUDA Graphs for better performance on repeated workloads
            "gpu_mem_limit": 4 * 1024 * 1024 * 1024,
        }),
    ]
)
eval_fn(session)

Evaluating ONNX model with providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']


Running ONNX Inference:   0%|          | 0/1000 [00:00<?, ?it/s]

Inference Time Statistics (milliseconds):
  Mean: 1.504121
  Std: 2.448048
  Min: 1.380869
  Max: 78.874833
  Median: 1.435211
  95th percentile: 1.463560
  99th percentile: 1.487714

Throughput: 664.84 samples/sec


#### TensorRT:

Cần cài TensorRT độc lập, không có quyền sudo để cài.

In [ ]:
session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session = ort.InferenceSession(
    ONNX_PATH, 
    sess_options=session_options, 
    providers = ['TensorRTExecutionProvider', 'CUDAExecutionProvider']
)
eval_fn(session)

: 

TensorRT:

ONNX model -> Convert to TensorRT engine -> inference on TensorRT Runtime

In [12]:
import tensorrt as trt
import os

TRT_LOGGER = trt.Logger(trt.Logger.INFO)

def build_engine(
    onnx_path,
    engine_path,
    fp16=True,
    int8=False,
    workspace_size=4 << 30  # 4GB
):
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(
        1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    )
    parser = trt.OnnxParser(network, TRT_LOGGER)

    config = builder.create_builder_config()
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, workspace_size)

    if fp16:
        config.set_flag(trt.BuilderFlag.FP16)

    if int8:
        config.set_flag(trt.BuilderFlag.INT8)
        # cần calibrator nếu INT8

    # Parse ONNX
    with open(onnx_path, "rb") as f:
        if not parser.parse(f.read()):
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            raise RuntimeError("Failed to parse ONNX")

    # Build engine
    serialized_engine = builder.build_serialized_network(network, config)
    if serialized_engine is None:
        raise RuntimeError("Engine build failed")

    with open(engine_path, "wb") as f:
        f.write(serialized_engine)

    print("✅ Engine saved:", engine_path)

build_engine(
    "resnet50.onnx",
    "resnet50.engine",
    fp16=True,
    workspace_size=4 << 30
)

[02/12/2026-16:01:18] [TRT] [W] Unable to determine GPU memory usage: In getGpuMemStatsInBytes at /_src/common/extended/resources.cpp:1180
[02/12/2026-16:01:18] [TRT] [E] createInferBuilder: Error Code 6: API Usage Error (CUDA initialization failure with error: 35. Please check your CUDA installation: http://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html In operator() at /_src/optimizer/api/builder.cpp:1399)
[02/12/2026-16:01:18] [TRT] [E] [checkMacros.cpp::catchCudaError::229] Error Code 1: Cuda Runtime (In catchCudaError at /_src/common/dispatch/checkMacros.cpp:229)


TypeError: pybind11::init(): factory function returned nullptr